<a href="https://colab.research.google.com/github/baronase/ml-residual-pendulum/blob/master/notebooks/pendulum_residual_rl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install "gymnasium[classic-control]" stable-baselines3 torch numpy matplotlib

In [ ]:
# --- REPO SETUP --- Only run once
!git clone https://github.com/baronase/ml-residual-pendulum.git

In [2]:
%cd ml-residual-pendulum
!git pull

/content/ml-residual-pendulum
Already up to date.


In [3]:
import sys
!{sys.executable} -m pip uninstall -y pendulum_residual_rl
!{sys.executable} -m pip install -e . --no-deps


Found existing installation: pendulum_residual_rl 0.1.0
Uninstalling pendulum_residual_rl-0.1.0:
  Successfully uninstalled pendulum_residual_rl-0.1.0
Obtaining file:///content/ml-residual-pendulum
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for pendulum_residual_rl (pyproject.toml) ... done
  Created wheel for pendulum_residual_rl: filename=pendulum_residual_rl-0.1.0-0.editable-py3-none-any.whl size=1319 sha256=37c392324a3b8f920bbc96c08f2082ef20a250ab2d199a2b3865093bc2ef4b94
  Stored in directory: /tmp/pip-ephem-wheel-cache-lr7zho5b/wheels/60/f4/4f/d02c1d5f55036909f72c6126ce700a1c121f66bb9d9c603732
Successfully built pendulum_residual_rl


In [4]:
#verifying import
import pendulum_residual_rl
print("ok", pendulum_residual_rl.__file__)

ok /content/ml-residual-pendulum/src/pendulum_residual_rl/__init__.py


In [ ]:
import gymnasium as gym
import numpy as np

env = gym.make("Pendulum-v1")
obs, info = env.reset(seed=0)

print("obs:", obs)
print("obs shape:", obs.shape)
print("action space:", env.action_space)
print("obs space:", env.observation_space)

# take a few random steps
for i in range(5):
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    print(i, "action:", action, "reward:", reward)


obs: [ 0.6520163   0.758205   -0.46042657]
obs shape: (3,)
action space: Box(-2.0, 2.0, (1,), float32)
obs space: Box([-1. -1. -8.], [1. 1. 8.], (3,), float32)
0 action: [-0.5652854] reward: -0.7620748568468063
1 action: [1.3246343] reward: -0.7443836493898092
2 action: [1.0806912] reward: -0.8761252845586246
3 action: [1.9851218] reward: -1.1987946893045602
4 action: [-1.0575279] reward: -1.82021578872846


In [ ]:
import math

def obs_to_theta_theta_dot(obs):
    cos_t, sin_t, theta_dot = obs
    theta = math.atan2(sin_t, cos_t)  # in [-pi, pi]
    return theta, float(theta_dot)

theta, theta_dot = obs_to_theta_theta_dot(obs)
print("theta:", theta, "theta_dot:", theta_dot)


theta: 1.250038370404206 theta_dot: 2.971787452697754
